In [31]:
import pandas as pd
from Main import settings, features, pf_set
import time
import pickle
from pandas.tseries.offsets import DateOffset
import Prepare_Data
import data_run_files
import return_prediction_functions
from return_prediction_functions import rff
import numpy as np
from pandas.tseries.offsets import MonthEnd
from sklearn.linear_model import Ridge
import General_Functions
import pickle
import Estimate_Covariance_Matrix
import os
import prepare_portfolio_data
import portfolio_choice_functions

start_date = pd.to_datetime('1952-12-31')

#Relevante paths
output_path = "./data_fifty/"
pickle_file_path = "./data_fifty/model_1.pkl"
wealth_path = "./data_fifty/wealth.csv"
barra_file_path = "./data_fifty/barra_cov.pkl"
output_path_market_returns = "data_fifty/market_returns_test.csv"

risk_free_path = "./data_fifty/risk_free_test.csv" 
output_data_ret_csv = "./data_fifty/data_ret.csv"  
output_data_ret_ld1_csv = "./data_fifty/data_ret_ld1.csv"

file_path_cluster_labels = "Data/Cluster Labels.csv"
file_path_factor_details = "Data/Factor Details.xlsx"

risk_free = data_run_files.process_risk_free_rate(risk_free_path, start_date)
daily_path = "./data_fifty/daily.csv"
chars_path = "./data_fifty/chars_behandlet.parquet"


Filtrering udført. Antal rækker efter filtrering: 841
Fil gemt som ./data_test/risk_free_test.csv


# Indlæser wealth, chars, daily, data_ret og data_ret_ld1

In [32]:
wealth = pd.read_csv(wealth_path)
wealth['eom'] = pd.to_datetime(wealth['eom'])
data_ret = pd.read_csv(output_data_ret_csv)
data_ret_ld1 = pd.read_csv(output_data_ret_ld1_csv)
data_ret['eom'] = pd.to_datetime(data_ret['eom'])
data_ret_ld1['eom'] = pd.to_datetime(data_ret_ld1['eom'])
data_ret_ld1['eom_ret'] = pd.to_datetime(data_ret_ld1['eom_ret'])
daily = pd.read_csv(daily_path, parse_dates=["date", "eom"])
chars = pd.read_parquet(chars_path)

Disse er checket og se ser korrekte ud wuhu :D

In [33]:
data_ret

,id,eom,ret_ld1,ret_ld2,ret_ld3,ret_ld4,ret_ld5,ret_ld6,ret_ld7,ret_ld8,ret_ld9,ret_ld10,ret_ld11,ret_ld12
0,10104,1986-04-30,-0.064934,0.031988,-0.369114,0.059902,-0.141147,0.258949,0.190520,-0.039651,0.297054,0.421617,0.137845,0.063897
1,10104,1986-05-31,0.031988,-0.369114,0.059902,-0.141147,0.258949,0.190520,-0.039651,0.297054,0.421617,0.137845,0.063897,0.155814
2,10104,1986-06-30,-0.369114,0.059902,-0.141147,0.258949,0.190520,-0.039651,0.297054,0.421617,0.137845,0.063897,0.155814,-0.188517
3,10104,1986-07-31,0.059902,-0.141147,0.258949,0.190520,-0.039651,0.297054,0.421617,0.137845,0.063897,0.155814,-0.188517,0.018414
4,10104,1986-08-31,-0.141147,0.258949,0.190520,-0.039651,0.297054,0.421617,0.137845,0.063897,0.155814,-0.188517,0.018414,0.171080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90698,92655,2022-07-31,-0.044328,-0.026268,0.096893,-0.016168,-0.032461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
90699,92655,2022-08-31,-0.026268,0.096893,-0.016168,-0.032461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
90700,92655,2022-09-30,0.096893,-0.016168,-0.032461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
90701,92655,2022-10-31,-0.016168,-0.032461,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Hent Predictions :D

### Indlæsning af Pickle fil (predictions)

In [34]:
with open(pickle_file_path, "rb") as file:
    model_1 = pickle.load(file)

#print(model_1) #Vi bruger dog ikke ikke denne i selve 

# Barra COV

Just a quick check

In [35]:
with open(barra_file_path, "rb") as file:
    barra_cov = pickle.load(file)

In [36]:
print(barra_cov['2022-11-30'])

{'fct_load':        BusEq  Chems  Durbl  Enrgy  Hlth  Manuf  Money  NoDur  Other  Shops  \
id                                                                           
10104    1.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
10107    1.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
10138    0.0    0.0    0.0    0.0   0.0    0.0    1.0    0.0    0.0    0.0   
10696    1.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
11308    0.0    0.0    0.0    0.0   0.0    0.0    0.0    1.0    0.0    0.0   
...      ...    ...    ...    ...   ...    ...    ...    ...    ...    ...   
87299    1.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
87432    1.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    0.0    0.0   
87447    0.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    1.0    0.0   
87717    0.0    0.0    0.0    0.0   0.0    0.0    0.0    0.0    1.0    0.0   
92655    0.0    0.0    0.0    0.0   0.0    0.0    1

In [37]:
print(barra_cov['2022-11-30']['ivol_vec'])

id
10104    0.004043
10107    0.002393
10138    0.002996
10696    0.003696
11308    0.001280
           ...   
87299    0.004106
87432    0.003534
87447    0.003139
87717    0.003166
92655    0.002555
Name: res_vol, Length: 180, dtype: float64


# Prepare Portfolio Data - tilføj return predictions, lambda list og datoer

In [38]:
chars = prepare_portfolio_data.add_return_predictions(chars,settings, output_path)

In [39]:
print(chars.shape) #Skal være 73216 x 131
print(chars[['id', 'eom','pred_ld1']])

(73216, 131)
          id        eom  pred_ld1
0      10104 1986-12-31       NaN
1      10104 1987-01-31       NaN
2      10104 1987-02-28       NaN
3      10104 1987-03-31       NaN
4      10104 1987-04-30       NaN
...      ...        ...       ...
73211  92655 2022-07-31  0.010957
73212  92655 2022-08-31  0.009506
73213  92655 2022-09-30  0.010641
73214  92655 2022-10-31  0.010774
73215  92655 2022-11-30  0.011608

[73216 rows x 3 columns]


In [40]:
lambda_list = prepare_portfolio_data.create_lambda_list(chars)

In [41]:
print(lambda_list['2021-10-31'])

{10104: 2.1068747835833751e-10, 10107: 3.034783408033585e-11, 10138: 1.014425423141795e-09, 10696: 4.3268657868458776e-10, 11308: 2.5674127083269843e-10, 11850: 1.5210112818417197e-10, 11955: 9.846108938145744e-10, 12060: 2.392171855561806e-10, 12062: 1.1115228863141125e-09, 12490: 3.103196145302579e-10, 13688: 1.2451492409136745e-09, 13856: 3.0647710531490904e-10, 13901: 5.984566822586187e-10, 14008: 3.4654779374858333e-10, 14277: 5.572628200703795e-10, 14541: 1.8678925436727748e-10, 14593: 1.7957730786929064e-11, 14702: 1.9775531529792074e-10, 15720: 1.868244462399027e-09, 17778: 3.3642773554010653e-10, 18163: 1.9283146734748652e-10, 18411: 7.948401354253297e-10, 19393: 3.2583660018022594e-10, 19561: 8.240904270111589e-11, 20482: 3.1099118409527434e-10, 21371: 1.314659590664529e-09, 21936: 1.611785023727204e-10, 22103: 8.191041680175024e-10, 22111: 1.9039009050878924e-10, 22592: 4.542098255993858e-10, 22752: 2.035214983043305e-10, 25320: 1.6935230409360711e-09, 25785: 1.9152306625614

In [42]:
first_cov_date, hp_years, start_oos = prepare_portfolio_data.define_important_dates(barra_cov, settings)
date_ranges = prepare_portfolio_data.create_date_ranges(settings, first_cov_date, start_oos, hp_years)
dates_m1 = date_ranges["dates_m1"]
dates_m2 = date_ranges["dates_m2"]
dates_oos = date_ranges["dates_oos"]
dates_hp = date_ranges["dates_hp"]
#Disse er alle korrekte

# Base cases

In [47]:
print(wealth['eom'])
print(dates_oos)
print(pf_set)

0     1952-11-30
1     1952-12-31
2     1953-01-31
3     1953-02-28
4     1953-03-31
         ...    
837   2022-08-31
838   2022-09-30
839   2022-10-31
840   2022-11-30
841   2022-12-31
Name: eom, Length: 842, dtype: datetime64[ns]
DatetimeIndex(['2010-12-31', '2011-01-31', '2011-02-28', '2011-03-31',
               '2011-04-30', '2011-05-31', '2011-06-30', '2011-07-31',
               '2011-08-31', '2011-09-30',
               ...
               '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31',
               '2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30',
               '2022-10-31', '2022-11-30'],
              dtype='datetime64[ns]', length=144, freq='ME')
{'wealth': 10000000000.0, 'gamma_rel': 10, 'mu': 0.007, 'lb_hor': 11}


In [48]:
ew = portfolio_choice_functions.ew_implement(data=chars, wealth=wealth, dates=dates_oos, pf_set=pf_set)
mkt = portfolio_choice_functions.mkt_implement(data=chars, wealth=wealth, dates=dates_oos, pf_set= pf_set)
rw = portfolio_choice_functions.rw_implement(data=chars, wealth=wealth, dates=dates_oos, pf_set= pf_set)
mv = portfolio_choice_functions.mv_implement(data=chars, cov_list=barra_cov ,wealth=wealth, dates=dates_oos, pf_set= pf_set)
factor_ml = portfolio_choice_functions.factor_ml_implement(data = chars, wealth = wealth, dates= dates_oos, n_pfs=settings["factor_ml"]["n_pfs"]
, gam=pf_set['gamma_rel'])

In [49]:
ew

{'w':           id        eom   w_start         w    tr_ld1    mu_ld1
 0      10104 2010-12-31  0.019620  0.005556       NaN       NaN
 1      10104 2011-01-31  0.005582  0.005556  0.024954  0.020052
 2      10104 2011-02-28  0.005493  0.005556  0.027140  0.038821
 3      10104 2011-03-31  0.005627  0.005556  0.016218  0.003383
 4      10104 2011-04-30  0.005825  0.005556  0.077442  0.027662
 ...      ...        ...       ...       ...       ...       ...
 25859  92655 2022-07-31  0.005393  0.005587  0.055684  0.093630
 25860  92655 2022-08-31  0.005544  0.005587 -0.043528 -0.036188
 25861  92655 2022-09-30  0.005999  0.005587 -0.024368 -0.091370
 25862  92655 2022-10-31  0.005677  0.005556  0.098793  0.081350
 25863  92655 2022-11-30  0.005211  0.005556 -0.013868  0.051329
 
 [25864 rows x 6 columns],
 'pf':      inv  shorting  turnover         r        tc    eom_ret type
 0    1.0       0.0  0.848747  0.022582  0.006040 2011-01-31  1/N
 1    1.0       0.0  0.052598  0.037496  0.00005

In [36]:
mkt

{'w':           id        eom   w_start         w    tr_ld1    mu_ld1
 0      10104 2010-12-31  0.019620  0.005556       NaN       NaN
 1      10104 2011-01-31  0.005582  0.005556  0.024954  0.020052
 2      10104 2011-02-28  0.005493  0.005556  0.027140  0.038821
 3      10104 2011-03-31  0.005627  0.005556  0.016218  0.003383
 4      10104 2011-04-30  0.005825  0.005556  0.077442  0.027662
 ...      ...        ...       ...       ...       ...       ...
 25859  92655 2022-07-31  0.005393  0.005587  0.055684  0.093630
 25860  92655 2022-08-31  0.005544  0.005587 -0.043528 -0.036188
 25861  92655 2022-09-30  0.005999  0.005587 -0.024368 -0.091370
 25862  92655 2022-10-31  0.005677  0.005556  0.098793  0.081350
 25863  92655 2022-11-30  0.005211  0.005556 -0.013868  0.051329
 
 [25864 rows x 6 columns],
 'pf':      inv  shorting  turnover         r        tc    eom_ret type
 0    1.0       0.0  0.848747  0.022582  0.006040 2011-01-31  1/N
 1    1.0       0.0  0.052598  0.037496  0.00005

In [37]:
rw

{'w':           id        eom   w_start         w    tr_ld1    mu_ld1
 0      10104 2010-12-31  0.019620  0.000123       NaN       NaN
 1      10104 2011-01-31  0.000124 -0.007284  0.024954  0.020052
 2      10104 2011-02-28 -0.007202  0.001111  0.027140  0.038821
 3      10104 2011-03-31  0.001125 -0.010000  0.016218  0.003383
 4      10104 2011-04-30 -0.010484 -0.008272  0.077442  0.027662
 ...      ...        ...       ...       ...       ...       ...
 25859  92655 2022-07-31 -0.008677 -0.005243  0.055684  0.093630
 25860  92655 2022-08-31 -0.005204 -0.011486 -0.043528 -0.036188
 25861  92655 2022-09-30 -0.012333 -0.006242 -0.024368 -0.091370
 25862  92655 2022-10-31 -0.006343 -0.006296  0.098793  0.081350
 25863  92655 2022-11-30 -0.005906 -0.001605 -0.013868  0.051329
 
 [25864 rows x 6 columns],
 'pf':      inv  shorting  turnover         r        tc    eom_ret     type
 0    2.0       1.0  2.519567  0.015278  0.077693 2011-01-31  Rank-ML
 1    2.0       1.0  0.826170  0.006433 

In [31]:
mv

{'w':           id        eom   w_start         w    tr_ld1    mu_ld1
 0      10104 2010-12-31  0.019620  0.017280       NaN       NaN
 1      10104 2011-01-31  0.017363  0.022978  0.024954  0.020052
 2      10104 2011-02-28  0.022719  0.019329  0.027140  0.038821
 3      10104 2011-03-31  0.019576  0.014793  0.016218  0.003383
 4      10104 2011-04-30  0.015510  0.024127  0.077442  0.027662
 ...      ...        ...       ...       ...       ...       ...
 25859  92655 2022-07-31 -0.014596 -0.010913  0.055684  0.093630
 25860  92655 2022-08-31 -0.010830 -0.006496 -0.043528 -0.036188
 25861  92655 2022-09-30 -0.006975  0.005305 -0.024368 -0.091370
 25862  92655 2022-10-31  0.005390 -0.000461  0.098793  0.081350
 25863  92655 2022-11-30 -0.000433 -0.008272 -0.013868  0.051329
 
 [25864 rows x 6 columns],
 'pf':           inv  shorting  turnover         r        tc    eom_ret  \
 0    2.439181  0.719590  2.475996 -0.038908  0.103482 2011-01-31   
 1    2.460849  0.730424  1.173957  0.0454

In [40]:
factor_ml

{'w':           id        eom  w_start    w    tr_ld1    mu_ld1
 0      10104 2010-12-31  0.01962  0.0       NaN       NaN
 1      10104 2011-01-31  0.00000  0.0  0.024954  0.020052
 2      10104 2011-02-28  0.00000  0.0  0.027140  0.038821
 3      10104 2011-03-31  0.00000  0.0  0.016218  0.003383
 4      10104 2011-04-30  0.00000  0.0  0.077442  0.027662
 ...      ...        ...      ...  ...       ...       ...
 25859  92655 2022-07-31  0.00000  0.0  0.055684  0.093630
 25860  92655 2022-08-31  0.00000  0.0 -0.043528 -0.036188
 25861  92655 2022-09-30  0.00000  0.0 -0.024368 -0.091370
 25862  92655 2022-10-31  0.00000  0.0  0.098793  0.081350
 25863  92655 2022-11-30  0.00000  0.0 -0.013868  0.051329
 
 [25864 rows x 6 columns],
 'pf':      inv  shorting  turnover         r        tc    eom_ret       type
 0    2.0       1.0  2.924576  0.017222  0.154048 2011-01-31  Factor-ML
 1    2.0       1.0  1.022224  0.018595  0.074021 2011-02-28  Factor-ML
 2    2.0       1.0  1.455416  0.009

In [41]:
tpf = portfolio_choice_functions.tpf_implement(data=chars, cov_list=barra_cov, wealth=wealth, dates=dates_oos, gam=pf_set["gamma_rel"])

In [43]:
tpf

{'w':           id        eom   w_start         w    tr_ld1    mu_ld1
 0      10104 2010-12-31  0.019620  0.015546       NaN       NaN
 1      10104 2011-01-31  0.015621  0.017275  0.024954  0.020052
 2      10104 2011-02-28  0.017081  0.022007  0.027140  0.038821
 3      10104 2011-03-31  0.022288  0.006919  0.016218  0.003383
 4      10104 2011-04-30  0.007254  0.017548  0.077442  0.027662
 ...      ...        ...       ...       ...       ...       ...
 25859  92655 2022-07-31  0.007726  0.034372  0.055684  0.093630
 25860  92655 2022-08-31  0.034111  0.024339 -0.043528 -0.036188
 25861  92655 2022-09-30  0.026134  0.075025 -0.024368 -0.091370
 25862  92655 2022-10-31  0.076235  0.059830  0.098793  0.081350
 25863  92655 2022-11-30  0.056120  0.040740 -0.013868  0.051329
 
 [25864 rows x 6 columns],
 'pf':           inv  shorting  turnover         r        tc    eom_ret          type
 0    2.374874  0.911181  2.749611 -0.019087  0.114453 2011-01-31  Markowitz-ML
 1    2.352451  0.88

# STATIC ML

# Predictions og Barra Cov (Ikke kør denne del)

Predictions

In [ ]:
search_grid_single = pd.DataFrame({
    'name': ['m1'],
    'horizon': [1]
})
search_grid = search_grid_single

In [ ]:
start_time = time.time()
models = []  # Liste til at gemme output for hver horizon

# Iterer over rækkerne i search_grid
for i in range(len(search_grid)):
    # Forbered y-variablen:
    h = search_grid.iloc[i]["horizon"]  # fx 1
    # Konstruer kolonnenavnet, fx "ret_ld1"
    col_name = "ret_ld" + str(h)
    # Beregn gennemsnittet pr. række – hvis der kun er én kolonne, svarer det til kolonneværdierne
    pred_y_values = data_ret[col_name]  # antages at være en Pandas Series
    # Opret en DataFrame med de nødvendige kolonner: id, eom
    pred_y_df = data_ret[['id', 'eom']].copy()
    # Beregn eom_pred_last: Vi antager, at vi ønsker at beregne slutdatoen for forudsigelsen som:
    # eom + (h+1) måneder - 1 dag (svarer nogenlunde til R's eom+1+months(max(h))-1)
    pred_y_df['eom_pred_last'] = pred_y_df['eom'] + MonthEnd(1)
    pred_y_df['ret_pred'] = pred_y_values

    # Join med de rækker i 'chars', hvor valid er True, på kolonnerne id og eom
    valid_chars = chars[chars['valid'] == True]
    data_pred = pd.merge(pred_y_df, valid_chars, on=['id', 'eom'], how='inner')
    print("data_pred.head")
    print(data_pred.head())
    print("data_pred.tail")
    print(data_pred.tail())
    print("horizons:", [h])

    # Bestem validerings-slutdatoer og test-inc baseret på settings
    update_freq = settings['split']['model_update_freq']
    if update_freq == "once":
        val_ends = [settings['split']['train_end']]
        test_inc = 1000
    elif update_freq == "yearly":
        # Opret en liste af datoer med årligt interval
        val_ends = pd.date_range(start=settings['split']['train_end'],
                                 end=settings['split']['test_end'],
                                 freq='YE').to_pydatetime().tolist()
        test_inc = 1
    elif update_freq == "decade":
        start_date = pd.to_datetime(settings['split']['train_end'])
        end_date = pd.to_datetime(settings['split']['test_end'])
        val_ends = []
        current = start_date
        while current <= end_date:
            val_ends.append(current)
            current += pd.DateOffset(years=10)
        test_inc = 10
    else:
        raise ValueError("Ugyldig model_update_freq i settings.")

    op = {}  # Dictionary til at gemme modeloutput for hver val_end
    inner_start = time.time()
    # Iterer over hver validerings-slutdato
    for val_end in val_ends:
        print(val_end)
        train_test_val = return_prediction_functions.data_split(
            data_pred,
            type=update_freq,
            val_end=val_end,
            val_years=settings['split']['val_years'],
            train_start=settings['screens']['start'],
            train_lookback=settings['split']['train_lookback'],
            retrain_lookback=settings['split']['retrain_lookback'],
            test_inc=test_inc,
            test_end=settings['split']['test_end']
        )
        print("datasættet er tomt; hvis ja, stop loopet")
        if train_test_val["test"].empty:
            print("Test datasættet er tomt for valideringsperiode:", val_end, ". Stopper forudsigelser.")
            break
        print("Train data:")
        print(train_test_val["train"].head())
        print("\nValidation data:")
        print(train_test_val["val"].head())
        print("\nTrain full data:")
        print(train_test_val["train_full"].head())
        print("\nTest data:")
        print(train_test_val["test"].head())
        model_start = time.time()
        model_op = return_prediction_functions.rff_hp_search(
            train_test_val,
            feat=features,
            p_vec=settings['rff']['p_vec'],
            g_vec=settings['rff']['g_vec'],
            l_vec=settings['rff']['l_vec'],
            seed=settings['seed_no']
        )
        model_time = time.time() - model_start
        print("Model training time:", model_time, "seconds")
        op[val_end] = model_op
    inner_time = time.time() - inner_start
    print("Total time for current horizon:", inner_time, "seconds")

    # Gem model-output for den aktuelle horizon til en pickle-fil (svarer til R's saveRDS)
    model_filename = f"{output_path}/model_{h}.pkl"
    with open(model_filename, "wb") as f:
        pickle.dump(op, f)

    models.append(op)

total_time = time.time() - start_time
print("Total run time:", total_time, "seconds")

Barra cov

In [ ]:
# Hvis du ikke har brug for den fjerde returnerede værdi, kan du bruge en underscore
cluster_data_d, ind_factors, clusters, cluster_data_m  = Estimate_Covariance_Matrix.process_cluster_data(chars, daily, file_path_cluster_labels, file_path_factor_details)

fct_ret_est = Estimate_Covariance_Matrix.run_regressions_by_date(cluster_data_d, ind_factors, clusters)
fct_ret = Estimate_Covariance_Matrix.create_factor_returns(fct_ret_est)

w_cor = (0.5 ** (1 / settings['cov_set']['hl_cor'])) ** np.arange(settings['cov_set']['obs'], 0, -1)
w_var = (0.5 ** (1 / settings['cov_set']['hl_var'])) ** np.arange(settings['cov_set']['obs'], 0, -1)

fct_dates = np.sort(fct_ret["date"].unique())
min_calc_date = fct_dates[settings["cov_set"]["obs"] - 1] - pd.DateOffset(months=1)
calc_dates = np.sort(cluster_data_m.loc[cluster_data_m["eom"] >= min_calc_date, "eom"].unique())

factor_cov_est = Estimate_Covariance_Matrix.factor_cov_estimate(calc_dates, fct_dates, fct_ret, w_cor, w_var, settings)

spec_risk = Estimate_Covariance_Matrix.unnest_spec_risk(fct_ret_est)
spec_risk_res_vol = Estimate_Covariance_Matrix.calculate_ewma(spec_risk, settings)

spec_risk_m, spec_risk_res_vol = Estimate_Covariance_Matrix.process_spec_risk_m(fct_dates, spec_risk_res_vol)
barra_cov = Estimate_Covariance_Matrix.calculate_barra_cov(calc_dates, cluster_data_m, spec_risk_m, factor_cov_est)

with open('./data_fifty/barra_cov.pkl', 'wb') as f:
    pickle.dump(barra_cov, f)